In [13]:
import mindwave, time
from pprint import pprint
import socket,select
import time, datetime, sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pygame import mixer
from pathlib import Path
import random

In [2]:
secondsToSample = 30
Fs=128 
show=False
samplepoints = Fs*secondsToSample

In [25]:
print("Connecting")
headset = mindwave.Headset('/dev/tty.MindWaveMobile-SerialPo')
time.sleep(2)
print("Connected!")

Connecting
Connected!


In [4]:
def on_raw(headset, rawvalue):
    (eeg, attention) = (headset.raw_value, headset.attention)
    
    ts = time.time()
    data['timestamp'].append(ts)
    data['raw_value'].append(eeg)
    data['attention'].append(attention)

    global currentTimestamp
    global currentRawValue
    global currentAttention
    currentTimestamp = ts
    currentRawValue = eeg
    currentAttention = attention

In [20]:
# grab all the music
currentDirectory = Path('./data/hip_hop')
currentPattern = "*.wav"
songs = []
for currentFile in currentDirectory.glob(currentPattern):
    songs.append(str(currentFile))
currentDirectory = Path('./data/classical')
currentPattern = "*.wav"
for currentFile in currentDirectory.glob(currentPattern):
    songs.append(str(currentFile))
random.shuffle(songs)

In [6]:
def one_song (song, subject, session):
    
    if song[5] == 'h':
        filename = './data/eeg/' + 'h_' + subject + '_' + session + '_' + song[13:-4] + '.csv'
    else:
        filename = './data/eeg/' + 'c_' + subject + '_' + session + '_' + song[15:-4] + '.csv'
    
    try:
        while (headset.poor_signal > 5):
            print("Headset signal noisy %d. Adjust the headset and the earclip." % (headset.poor_signal))
            time.sleep(0.1)

        print("Writing %d seconds output to %s" % (secondsToSample,filename))
        stime = time.time()
        headset.raw_value_handlers.append( on_raw )
        prevTime = 0
        while ((time.time()-stime)<secondsToSample):
            if headset.poor_signal > 5 :
                print("Headset signal noisy %d. Adjust the headset and the earclip." % (headset.poor_signal))

            if currentTimestamp is not None: 
                sampled_data["timestamp"].append(currentTimestamp)
                sampled_data["raw_value"].append(currentRawValue)
                sampled_data["attention"].append(currentAttention)

            timeDiff = int(time.time()-stime)
            if(timeDiff != prevTime) : 
                print("seconds elapsed: " + str(timeDiff))
                prevTime = timeDiff
            time.sleep(1/Fs)
            pass

    finally:

        df = pd.DataFrame.from_dict(sampled_data)
        df.to_csv(filename, index=False)


In [26]:
subject = 'patrick'
session = '1'
for song in songs:
    mixer.init()
    mixer.music.load(song)
    mixer.music.play()
    data = dict()
    data['timestamp'] = list()
    data['raw_value'] = list()
    data['attention'] = list()
    currentTimestamp = None
    currentRawValue = None
    currentAttention = None
    sampled_data = dict()
    sampled_data['timestamp'] = list() 
    sampled_data['raw_value'] = list()
    sampled_data['attention'] = list()
    one_song(song, subject, session)
headset.stop()

Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25.

Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51.

Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51.

Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51.

Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51.

...

Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51.

Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51.

Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51. Adjust the headset and the earclip.
Headset signal noisy 51.

Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25. Adjust the headset and the earclip.
Headset signal noisy 25.

KeyboardInterrupt: 